# RL-CAI Training Notebook (Fixed for Stacked LoRA)

This notebook implements the Reinforcement Learning stage of Constitutional AI (RL-CAI) with proper support for stacked LoRA models.

Steps:
1. Generate preference data using SL-CAI models (HM7B + CAI LoRA)
2. Train reward models on constitutional preferences
3. Fine-tune with PPO using constitutional feedback

**Important**: Run the cells in order, starting with Section 0!

## 0. Prerequisites - Run This First!

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

# Set up paths
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Constitutional_AI_Project'
PROJECT_DIR = '/content/Constitutional_AI_Project'
GITHUB_REPO = 'https://github.com/ychleee/Constitutional_AI_Project.git'

# Clone or update repository
if not os.path.exists(PROJECT_DIR):
    print('📥 Cloning repository...')
    !git clone {GITHUB_REPO} {PROJECT_DIR}
else:
    print('📥 Updating repository...')
    !cd {PROJECT_DIR} && git pull origin main

# Add project to Python path
sys.path.append(PROJECT_DIR)

# Install required dependencies
print('📦 Installing dependencies...')
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers>=4.35.0 datasets>=2.14.0 accelerate>=0.24.0
!pip install -q peft>=0.6.0 trl>=0.7.0 bitsandbytes>=0.41.0
!pip install -q einops tensorboard wandb safetensors
!pip install -q jsonlines pandas numpy scikit-learn matplotlib seaborn tqdm rich

# Create necessary directories in Drive
directories = [
    f'{DRIVE_PROJECT_PATH}/data/red_team',
    f'{DRIVE_PROJECT_PATH}/data/helpfulness',
    f'{DRIVE_PROJECT_PATH}/data/sl_datasets',
    f'{DRIVE_PROJECT_PATH}/data/rl_datasets',
    f'{DRIVE_PROJECT_PATH}/models/hm7b',
    f'{DRIVE_PROJECT_PATH}/models/hm7b_deontological',
    f'{DRIVE_PROJECT_PATH}/models/hm7b_consequentialist',
    f'{DRIVE_PROJECT_PATH}/models/deontological/reward_model',
    f'{DRIVE_PROJECT_PATH}/models/deontological/rl_cai',
    f'{DRIVE_PROJECT_PATH}/models/consequentialist/reward_model',
    f'{DRIVE_PROJECT_PATH}/models/consequentialist/rl_cai',
    f'{DRIVE_PROJECT_PATH}/results/rl_training_logs'
]

for dir_path in directories:
    os.makedirs(dir_path, exist_ok=True)

print('✅ Prerequisites complete!')

## 1. Setup Environment and Configuration

In [ ]:
import os
import sys
import json
import torch
import gc
from pathlib import Path

# Paths
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Constitutional_AI_Project'
PROJECT_DIR = '/content/Constitutional_AI_Project'
DATA_PATH = f"{DRIVE_PROJECT_PATH}/data"
MODEL_PATH = f"{DRIVE_PROJECT_PATH}/models"

# Model paths for stacked LoRA architecture
BASE_MODEL_NAME = "mistralai/Mistral-7B-v0.1"  # Base Mistral model
HM7B_PATH = f"{MODEL_PATH}/hm7b"  # HM7B LoRA adapter
DEONT_CAI_PATH = f"{MODEL_PATH}/hm7b_deontological"  # Deontological CAI LoRA
CONSEQ_CAI_PATH = f"{MODEL_PATH}/hm7b_consequentialist"  # Consequentialist CAI LoRA

# Check GPU
if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    print(f"✅ GPU: {gpu_info.name}")
    print(f"   Memory: {gpu_info.total_memory / 1024**3:.2f} GB")
else:
    print("❌ No GPU detected!")

# Configuration based on GPU
if "A100" in gpu_info.name:
    CONFIG = {
        "batch_size": 4,
        "gradient_accumulation": 4,
        "max_length": 1024,
        "learning_rate": 1e-5,
        "ppo_batch_size": 8,
        "mini_batch_size": 2
    }
else:  # T4 or other GPUs
    CONFIG = {
        "batch_size": 1,
        "gradient_accumulation": 8,
        "max_length": 512,
        "learning_rate": 2e-5,
        "ppo_batch_size": 4,
        "mini_batch_size": 1
    }

print(f"\nConfiguration: {CONFIG}")

## 2. Helper Functions for Stacked LoRA Loading

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def load_stacked_lora_model(constitution_type, device_map="auto"):
    """
    Load a stacked LoRA model: Base Mistral -> HM7B LoRA -> CAI LoRA
    
    Args:
        constitution_type: 'deontological' or 'consequentialist'
        device_map: Device mapping for model
    
    Returns:
        model: The loaded model with stacked LoRA
        tokenizer: The tokenizer
    """
    print(f"🔄 Loading {constitution_type} model with stacked LoRA...")
    
    # Step 1: Load base Mistral model
    print("  Loading base Mistral-7B...")
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        trust_remote_code=True
    )
    
    # Step 2: Load and merge HM7B LoRA
    if os.path.exists(HM7B_PATH):
        print("  Loading HM7B LoRA adapter...")
        hm7b_model = PeftModel.from_pretrained(base_model, HM7B_PATH)
        print("  Merging HM7B LoRA...")
        merged_model = hm7b_model.merge_and_unload()
        del hm7b_model
        torch.cuda.empty_cache()
    else:
        print("  ⚠️ HM7B not found, using base model")
        merged_model = base_model
    
    # Step 3: Load CAI LoRA on top
    cai_path = DEONT_CAI_PATH if constitution_type == "deontological" else CONSEQ_CAI_PATH
    if os.path.exists(cai_path):
        print(f"  Loading {constitution_type} CAI LoRA adapter...")
        final_model = PeftModel.from_pretrained(merged_model, cai_path)
    else:
        print(f"  ⚠️ {constitution_type} CAI LoRA not found at {cai_path}")
        # Try alternative path (from SL training)
        alt_path = f"{MODEL_PATH}/{constitution_type}/sl_cai"
        if os.path.exists(alt_path):
            print(f"  Using SL-CAI model from {alt_path}")
            final_model = PeftModel.from_pretrained(merged_model, alt_path)
        else:
            print(f"  Using HM7B model without CAI LoRA")
            final_model = merged_model
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"✅ Loaded {constitution_type} model")
    return final_model, tokenizer

def clear_gpu_memory():
    """Clear GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        print(f"GPU memory: {torch.cuda.memory_allocated()/1024**3:.2f}GB used")

## 3. Generate Preference Data

In [ ]:
from src.data.preference_generator import PreferenceGenerator
import random
from tqdm import tqdm

def generate_preferences_for_constitution(constitution_type, num_prompts=20):
    """Generate preference data for a constitution using stacked LoRA model"""
    
    print(f"\n{'='*60}")
    print(f"🔄 Generating preferences for {constitution_type} model...")
    print(f"{'='*60}")
    
    # Load the stacked LoRA model
    model, tokenizer = load_stacked_lora_model(constitution_type)
    model.eval()
    
    # Initialize preference generator with a judge model
    # We can use the same model or a different one for judging
    pref_generator = PreferenceGenerator(
        feedback_model_name=BASE_MODEL_NAME,  # Use base model for judging
        constitution_path=f"{PROJECT_DIR}/constitutions/{constitution_type}/principles.json",
        constitution_type=constitution_type,
        use_soft_labels=True,
        min_score_difference=0.1
    )
    
    # Load test prompts
    red_team_path = f"{DATA_PATH}/red_team/sample_red_team.json"
    if not os.path.exists(red_team_path):
        print("❌ Red team data not found!")
        return None
    
    with open(red_team_path, 'r') as f:
        red_team_data = json.load(f)
    
    prompts = [item['prompt'] for item in red_team_data['prompts'][:num_prompts]]
    print(f"Using {len(prompts)} red team prompts")
    
    # Generate multiple responses per prompt
    def generate_response(prompt):
        """Generate a response using the model"""
        formatted = f"Human: {prompt}\n\nAssistant:"
        inputs = tokenizer(formatted, return_tensors="pt", truncation=True, max_length=512)
        
        # Move inputs to GPU if available
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            # Vary temperature for diversity
            temperature = 0.7 + random.random() * 0.4
            
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response.split("Assistant:")[-1].strip()
    
    # Generate preference pairs
    print("Generating responses and preference pairs...")
    all_pairs = pref_generator.process_dataset(
        prompts=prompts,
        response_generator=generate_response,
        responses_per_prompt=4,
        show_progress=True
    )
    
    # Save preference data
    output_path = f"{DATA_PATH}/rl_datasets/{constitution_type}_preferences.jsonl"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    pref_generator.save_preferences(all_pairs, output_path)
    
    print(f"✅ Generated {len(all_pairs)} preference pairs")
    print(f"   Saved to: {output_path}")
    
    # Clean up
    del model
    clear_gpu_memory()
    
    return output_path

# Generate for both constitutions
deont_pref_path = generate_preferences_for_constitution("deontological", num_prompts=10)
conseq_pref_path = generate_preferences_for_constitution("consequentialist", num_prompts=10)

print("\n✅ Preference generation complete!")

## 4. Train Reward Models

In [ ]:
from src.training.reward_model import RewardModelTrainer

def train_reward_model(constitution_type, pref_data_path):
    """Train reward model for a constitution"""
    
    print(f"\n{'='*60}")
    print(f"🎯 Training reward model for {constitution_type}...")
    print(f"{'='*60}")
    
    output_dir = f"{MODEL_PATH}/{constitution_type}/reward_model"
    
    # Initialize trainer
    trainer = RewardModelTrainer(
        model_name=BASE_MODEL_NAME,
        constitution_type=constitution_type,
        output_dir=output_dir,
        use_soft_labels=True
    )
    
    # Train
    trainer.train(
        train_data_path=pref_data_path,
        val_data_path=None,  # Could split data for validation
        epochs=2,  # Fewer epochs for reward model
        batch_size=CONFIG['batch_size'],
        learning_rate=CONFIG['learning_rate']
    )
    
    print(f"✅ Reward model saved to {output_dir}")
    
    # Clean up
    clear_gpu_memory()
    
    return output_dir

# Train reward models if preference data exists
if deont_pref_path and os.path.exists(deont_pref_path):
    deont_reward_path = train_reward_model("deontological", deont_pref_path)
else:
    print("⚠️ Skipping deontological reward model training - no preference data")
    deont_reward_path = None

if conseq_pref_path and os.path.exists(conseq_pref_path):
    conseq_reward_path = train_reward_model("consequentialist", conseq_pref_path)
else:
    print("⚠️ Skipping consequentialist reward model training - no preference data")
    conseq_reward_path = None

print("\n✅ Reward model training complete!")

## 5. PPO Training with TRL (Updated for 0.7.0+)

In [ ]:
from trl import PPOTrainer, PPOConfig
from trl.models import AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

def train_ppo_with_stacked_lora(constitution_type, reward_model_path):
    """Train model with PPO using constitutional rewards and stacked LoRA"""
    
    print(f"\n{'='*60}")
    print(f"🚀 PPO training for {constitution_type} model...")
    print(f"{'='*60}")
    
    # PPO configuration for TRL 0.7.0+
    ppo_config = PPOConfig(
        batch_size=CONFIG['ppo_batch_size'],
        mini_batch_size=CONFIG['mini_batch_size'],
        gradient_accumulation_steps=CONFIG['gradient_accumulation'],
        ppo_epochs=4,
        learning_rate=CONFIG['learning_rate'],
        optimize_cuda_cache=True,
        seed=42,
        use_score_scaling=True,
        use_score_norm=True,
        score_clip=10.0,
    )
    
    # Load the stacked LoRA model
    print("Loading stacked LoRA model for PPO...")
    base_model, tokenizer = load_stacked_lora_model(constitution_type)
    
    # Wrap model with value head for PPO
    # Note: For stacked LoRA, we need to be careful about the value head
    print("Adding value head for PPO...")
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        base_model,
        trust_remote_code=True
    )
    
    # Load reward model
    print("Loading reward model...")
    from src.training.reward_model import RewardModel
    reward_model = RewardModel(BASE_MODEL_NAME)
    
    reward_checkpoint = f"{reward_model_path}/final_model/reward_model.pt"
    if os.path.exists(reward_checkpoint):
        reward_model.load_state_dict(torch.load(reward_checkpoint, map_location="cpu"))
        reward_model.eval()
        if torch.cuda.is_available():
            reward_model.cuda()
    else:
        print(f"⚠️ Reward model checkpoint not found at {reward_checkpoint}")
        print("   Using random reward model (not recommended!)")
    
    # Create PPO trainer
    ppo_trainer = PPOTrainer(
        config=ppo_config,
        model=model,
        tokenizer=tokenizer
    )
    
    # Load training prompts
    with open(f"{DATA_PATH}/red_team/sample_red_team.json", 'r') as f:
        red_team_data = json.load(f)
    
    prompts = [item['prompt'] for item in red_team_data['prompts'][:30]]  # Use 30 prompts
    print(f"Using {len(prompts)} prompts for PPO training")
    
    # Training loop
    print("Starting PPO training...")
    for epoch in range(2):  # 2 epochs
        print(f"\nEpoch {epoch + 1}/2")
        epoch_rewards = []
        
        with tqdm(range(0, len(prompts), ppo_config.batch_size), desc="PPO Steps") as pbar:
            for batch_start in pbar:
                batch_prompts = prompts[batch_start:batch_start + ppo_config.batch_size]
                
                # Format prompts
                formatted_prompts = [f"Human: {p}\n\nAssistant:" for p in batch_prompts]
                
                # Tokenize
                inputs = tokenizer(
                    formatted_prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=CONFIG['max_length'] // 2
                )
                
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                # Generate responses
                with torch.no_grad():
                    response_ids = model.generate(
                        **inputs,
                        max_new_tokens=150,
                        do_sample=True,
                        temperature=0.7,
                        pad_token_id=tokenizer.pad_token_id
                    )
                
                # Extract response tokens only
                response_ids = response_ids[:, inputs['input_ids'].shape[1]:]
                
                # Get responses text for reward computation
                responses = tokenizer.batch_decode(response_ids, skip_special_tokens=True)
                
                # Compute rewards using reward model
                rewards = []
                for prompt, response in zip(formatted_prompts, responses):
                    # Format for reward model
                    full_text = f"{prompt}{response}"
                    encoding = tokenizer(
                        full_text,
                        return_tensors="pt",
                        truncation=True,
                        max_length=CONFIG['max_length']
                    )
                    
                    if torch.cuda.is_available():
                        encoding = {k: v.cuda() for k, v in encoding.items()}
                    
                    with torch.no_grad():
                        reward_output = reward_model(
                            input_ids=encoding['input_ids'],
                            attention_mask=encoding['attention_mask']
                        )
                        reward = reward_output['logits'].squeeze().item()
                    
                    rewards.append(reward)
                
                # Convert rewards to tensor
                rewards_tensor = torch.tensor(rewards)
                if torch.cuda.is_available():
                    rewards_tensor = rewards_tensor.cuda()
                
                # PPO step
                stats = ppo_trainer.step(
                    queries=inputs['input_ids'],
                    responses=response_ids,
                    scores=rewards_tensor
                )
                
                # Track rewards
                epoch_rewards.extend(rewards)
                
                # Update progress bar
                pbar.set_postfix({
                    'reward': f"{np.mean(rewards):.3f}",
                    'epoch_avg': f"{np.mean(epoch_rewards):.3f}"
                })
        
        print(f"Epoch {epoch + 1} - Average Reward: {np.mean(epoch_rewards):.3f}")
    
    # Save model
    output_dir = f"{MODEL_PATH}/{constitution_type}/rl_cai"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Saving model to {output_dir}...")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    print(f"✅ RL-CAI model saved to {output_dir}")
    
    # Clean up
    del model
    del reward_model
    clear_gpu_memory()
    
    return output_dir

# Train both models with PPO
import numpy as np  # Make sure numpy is imported

if deont_reward_path:
    deont_rl_path = train_ppo_with_stacked_lora("deontological", deont_reward_path)
else:
    print("⚠️ Skipping deontological PPO training - no reward model")
    deont_rl_path = None

if conseq_reward_path:
    conseq_rl_path = train_ppo_with_stacked_lora("consequentialist", conseq_reward_path)
else:
    print("⚠️ Skipping consequentialist PPO training - no reward model")
    conseq_rl_path = None

print("\n✅ PPO training complete!")

## 6. Test RL-CAI Models

In [ ]:
def test_rl_model(model_path, constitution_type, prompt):
    """Test an RL-CAI trained model"""
    
    # Load model and tokenizer
    print(f"Loading {constitution_type} RL-CAI model...")
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # Format prompt
    formatted = f"Human: {prompt}\n\nAssistant:"
    inputs = tokenizer(formatted, return_tensors="pt", truncation=True)
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("Assistant:")[-1].strip()
    
    # Clean up
    del model
    clear_gpu_memory()
    
    return response

# Test on ethical dilemmas
test_prompts = [
    "Is it ever acceptable to lie to protect someone's feelings?",
    "Should I report a friend who is cheating on their taxes?",
    "Is it okay to break a promise if it helps more people?"
]

print("="*60)
print("RL-CAI MODEL TESTING")
print("="*60)

for test_prompt in test_prompts:
    print(f"\nPrompt: {test_prompt}")
    print("-"*60)
    
    if deont_rl_path and os.path.exists(deont_rl_path):
        print("\n🔵 Deontological RL-CAI:")
        deont_response = test_rl_model(deont_rl_path, "deontological", test_prompt)
        print(deont_response[:400])
    
    if conseq_rl_path and os.path.exists(conseq_rl_path):
        print("\n🟢 Consequentialist RL-CAI:")
        conseq_response = test_rl_model(conseq_rl_path, "consequentialist", test_prompt)
        print(conseq_response[:400])
    
    print("\n" + "="*60)

## 7. Save Training Summary

In [ ]:
import datetime

# Create training summary
rl_summary = {
    "training_date": datetime.datetime.now().isoformat(),
    "base_model": BASE_MODEL_NAME,
    "architecture": "Stacked LoRA: Base Mistral -> HM7B -> CAI",
    "gpu_type": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
    "stages_completed": [
        "preference_generation",
        "reward_model_training",
        "ppo_training"
    ],
    "models": {
        "deontological": {
            "hm7b_base": HM7B_PATH,
            "sl_cai": DEONT_CAI_PATH,
            "reward_model": deont_reward_path if 'deont_reward_path' in locals() else None,
            "rl_cai": deont_rl_path if 'deont_rl_path' in locals() else None,
            "preferences": deont_pref_path if 'deont_pref_path' in locals() else None
        },
        "consequentialist": {
            "hm7b_base": HM7B_PATH,
            "sl_cai": CONSEQ_CAI_PATH,
            "reward_model": conseq_reward_path if 'conseq_reward_path' in locals() else None,
            "rl_cai": conseq_rl_path if 'conseq_rl_path' in locals() else None,
            "preferences": conseq_pref_path if 'conseq_pref_path' in locals() else None
        }
    },
    "config": CONFIG
}

# Save summary
summary_path = f"{DRIVE_PROJECT_PATH}/results/rl_training_summary.json"
os.makedirs(os.path.dirname(summary_path), exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(rl_summary, f, indent=2)

print(f"✅ Training summary saved to {summary_path}")
print("\n🎉 RL-CAI training complete!")
print("\nNext steps:")
print("1. Run 03_evaluation_colab.ipynb to evaluate the models")
print("2. Compare responses on divergent ethical dilemmas")
print("3. Analyze chain-of-thought reasoning patterns")
print("\nKey paths:")
print(f"  Deontological RL-CAI: {deont_rl_path if 'deont_rl_path' in locals() else 'Not trained'}")
print(f"  Consequentialist RL-CAI: {conseq_rl_path if 'conseq_rl_path' in locals() else 'Not trained'}")